In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

import Metashape 

from easydict import EasyDict as edict

from lib.io import (write_markers_by_camera, 
                    write_markers_by_marker, 
                    write_marker_world_coordinates,
                    )

# from lib.metashape import metashape_workflow_functions as meta
# from lib.metashape import metashape_license_setup
# from lib.metashape.read_yaml import read_yaml

# ---- Checl if Metashape is activated
if Metashape.app.activated:
    print("Metashape is activated: ", Metashape.app.activated)
else:
    print("No licence found. Please check that you linked your license (floating or standalone) wih the Metashape python module.")
    exit()

## Define where to get the config file (only used if running interactively)
# cfg_path = "config/base.yml"
# cfg = edict(read_yaml(cfg_path))

Metashape is activated:  True


In [ ]:
load_project = "data/belpy.psx"
project_file = "data/belpy_new.psx"
# markers_file = "data/belpy_markers.csv"

# create a handle to the Metashape object
# When running via Metashape, can use: doc = Metashape.app.document
doc = Metashape.Document()

# If specified, open existing project
if load_project != "":
    doc.open(load_project)
else:
    # Initialize a chunk, set its CRS as specified
    chunk = doc.addChunk()
    # chunk.crs = Metashape.CoordinateSystem(cfg["project_crs"])
    # chunk.marker_crs = Metashape.CoordinateSystem(
    #     cfg["addGCPs"]["gcp_crs"])

# Save doc doc as new project (even if we opened an existing project, save as a separate one so the existing project remains accessible in its original state)
doc.save(project_file)

chunk = doc.chunk
write_markers_by_camera(chunk, "data/belpy2807_markers_images.csv", 
                        convert_to_micron=True)
write_marker_world_coordinates(chunk, "data/belpy2807_markers_world.csv")
